In [2]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import neptune.new as neptune
import alphatims.bruker as bruker

from alphabase.spectral_library.base import SpecLibBase
from alphadia.extraction import data, planning, workflow
from alphadia.extraction.workflow import manager, peptidecentric



raw_files = [
    '/Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d',
]
output_location = '/Users/georgwallmann/Documents/data/performance_tests/bruker/outputs'

/var/folders/lc/9594t94d5b5_gn0y04w1jh980000gn/T/ipykernel_64621/1313159942.py:8: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  import neptune.new as neptune


In [3]:
speclib = '/Users/georgwallmann/Documents/data/alphadia_benchmarking/libraries/patricia_hela/21min_Evosep_HeLa_BR14_48fractions_diaPASEF_py_diAID_2_egs.hdf'
test_lib = SpecLibBase()
test_lib.load_hdf(speclib, load_mod_seq=True)

In [20]:
plan = planning.Plan(output_location, raw_files, test_lib)

plan.config['general']['reuse_calibration'] = False
plan.config['general']['save_figueres'] = False
plan.config['general']['thread_count'] = 10
plan.config['general']['astral_ms1'] = False
plan.config['library_loading']['channel_filter'] = '0'
plan.config['library_loading']['exclude_shared_ions'] = False
plan.config['calibration']['norm_rt_mode'] = 'linear'

plan.config['extraction_target']['target_num_candidates'] = 3
plan.config['extraction_target']['target_ms1_tolerance'] = 30
plan.config['extraction_target']['target_ms2_tolerance'] = 30
plan.config['extraction_target']['target_rt_tolerance'] = 150

0:00:00.000107  PROGRESS:       _   _      _         ___ ___   _   
0:00:00.000593  PROGRESS:      /_\ | |_ __| |_  __ _|   \_ _| /_\  
0:00:00.001020  PROGRESS:     / _ \| | '_ \ ' \/ _` | |) | | / _ \ 
0:00:00.001327  PROGRESS:    /_/ \_\_| .__/_||_\__,_|___/___/_/ \_\
0:00:00.001741  PROGRESS:            |_|                            
0:00:00.002065  PROGRESS: 
0:00:00.002554  INFO: loading default config from /Users/georgwallmann/Documents/git/alphadia/alphadia/extraction/../../misc/config/default.yaml
0:00:00.010513  PROGRESS: version: 1.3.1
0:00:00.010932  PROGRESS: hostname: Georgs-MacBook-Pro.local
0:00:00.011269  PROGRESS: date: 2023-09-19 20:25:17
0:00:05.085849  INFO: renaming precursor_columns columns
0:00:05.086369  INFO: renaming fragment_columns columns
0:00:05.086891  INFO: ========= Library Stats =========
0:00:05.087216  INFO: Number of precursors: 460,153
0:00:05.151340  INFO: 	thereof targets:230,483
0:00:05.152083  INFO: 	thereof decoys: 229,670
0:00:05.155709  IN

In [21]:
for raw_name, dia_path, speclib in plan.get_run_data():
    pass

In [22]:
workflow = peptidecentric.PeptideCentricWorkflow(
    raw_name,
    plan.config,
    dia_path,
    speclib
)

https://app.neptune.ai/MannLabs/alphaDIA/e/AL-337


0:00:08.596494  INFO: Importing data from /Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d
0:00:08.597051  INFO: Using .d import for /Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d
0:00:08.597463  INFO: Reading frame metadata for /Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d
0:00:08.793820  INFO: Reading 17,438 frames with 1,635,017,608 detector events for /Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d
100%|██████████| 17438/17438 [00:07<00:00, 2446.79it/s]
0:00:16.016436  INFO: Indexing /Users/georgwallmann/Documents/data/testing/bruker/20230422_TIMS05_PaSk_MCT_SA_HeLa_mDIA_P001_diaP_pydiAID8_1300V_S2-E3_1_1819.d...
0:00:16.018792  INFO: Bruker DLL not available, e

In [29]:
test_df = workflow.spectral_library.precursor_df.sample(1000)

In [34]:
from alphadia.extraction import hybridselection

config = hybridselection.HybridCandidateConfig()
config.update(workflow.config['selection_config'])
config.update({
    'rt_tolerance':workflow.com.rt_error,
    'mobility_tolerance': workflow.com.mobility_error,
    'candidate_count': workflow.com.num_candidates,
    'precursor_mz_tolerance': workflow.com.ms1_error,
    'fragment_mz_tolerance': workflow.com.ms2_error,
    'exclude_shared_ions': workflow.config['library_loading']['exclude_shared_ions']
})

extraction = hybridselection.HybridCandidateSelection(
    workflow.dia_data.jitclass(),
    test_df,
    workflow.spectral_library.fragment_df,
    config.jitclass(),
    rt_column = f'rt_{workflow.com.column_type}',
    mobility_column = f'mobility_{workflow.com.column_type}',
    precursor_mz_column = f'mz_{workflow.com.column_type}',
    fragment_mz_column = f'mz_{workflow.com.column_type}',
    fwhm_rt = workflow.optimization_manager.fwhm_rt,
    fwhm_mobility = workflow.optimization_manager.fwhm_mobility
)
candidates_df = extraction(
    thread_count=workflow.config['general']['thread_count']
)

0:04:55.627185  INFO: Duty cycle consists of 9 frames, 0.96 seconds cycle time
0:04:55.627841  INFO: Duty cycle consists of 928 scans, 0.00065 1/K_0 resolution
0:04:55.628282  INFO: FWHM in RT is 5.00 seconds, sigma is 1.11
0:04:55.628570  INFO: FWHM in mobility is 0.010 1/K_0, sigma is 6.57
0:04:55.631108  INFO: Starting candidate selection


100%|██████████| 1000/1000 [00:10<00:00, 96.39it/s]
0:05:06.320512  INFO: Finished candidate selection


In [36]:
from alphadia.extraction import plexscoring

config = plexscoring.CandidateConfig()
config.update(workflow.config['scoring_config'])
config.update({
    'precursor_mz_tolerance': workflow.com.ms1_error,
    'fragment_mz_tolerance': workflow.com.ms2_error,
    'exclude_shared_ions': workflow.config['library_loading']['exclude_shared_ions']
})

candidate_scoring = plexscoring.CandidateScoring(
    workflow.dia_data.jitclass(),
    workflow.spectral_library._precursor_df,
    workflow.spectral_library._fragment_df,
    config = config,
    rt_column = f'rt_{workflow.com.column_type}',
    mobility_column = f'mobility_{workflow.com.column_type}',
    precursor_mz_column = f'mz_{workflow.com.column_type}',
    fragment_mz_column = f'mz_{workflow.com.column_type}',
)

features_df, fragments_df = candidate_scoring(
    candidates_df, 
    thread_count=workflow.config['general']['thread_count'],
)

0:05:06.624870  INFO: Starting candidate scoring
100%|██████████| 1000/1000 [00:00<00:00, 4020.27it/s]
0:05:07.001336  WARNING: delta_frame_peak has 6 NaNs ( 0.60 % out of 1000)
0:05:07.032154  INFO: Finished candidate scoring
